In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 20,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['merror'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['multi:softmax']),
    'eval_metric':hp.choice('eval_metric',[['merror']]),
    'num_class':hp.choice('num_class',[10]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.5988658235050284, 'colsample_bytree': 0.8896115495695454, 'eta': 0.00780174288166175, 'eval_metric': ('merror',), 'extra_dims': 7, 'gamma': 2.7616226083945422e-06, 'lambda': 4.926516375970026e-06, 'max_depth': 4, 'min_child_weight': 31.025627776682082, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.8043606730234718}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[10:35:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verifi

Setting param `disable_default_eval_metric` to 1.                      
[10:41:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.07817	test-merror:0.09000                           

[1]	train-merror:0.06440	test-merror:0.07180                           

[2]	train-merror:0.05783	test-merror:0.06740                           

[3]	train-merror:0.05437	test-merror:0.06270                           

[4]	train-merror:0.05238	test-merror:0.06060                           

[5]	train-merror:0.05125	test-merror:0.05890                           

[6]	train-merror:0.04938	test-merror:0.05880                           

[7]	train-merror:0.04885	test-merror:0.05840                  

[1]	train-merror:0.05450	test-merror:0.06980                           

[2]	train-merror:0.04695	test-merror:0.06280                           

[3]	train-merror:0.04333	test-merror:0.05910                           

[4]	train-merror:0.03912	test-merror:0.05540                           

[5]	train-merror:0.03650	test-merror:0.05400                           

[6]	train-merror:0.03435	test-merror:0.05220                           

[7]	train-merror:0.03210	test-merror:0.05030                           

[8]	train-merror:0.02957	test-merror:0.04830                           

[9]	train-merror:0.02770	test-merror:0.04660                           

[10]	train-merror:0.02572	test-merror:0.04550                          

[11]	train-merror:0.02388	test-merror:0.04400                          

[12]	train-merror:0.02237	test-merror:0.04280                          

[13]	train-merror:0.02102	test-merror:0.04120                          

[14]	train-merror:0.01953	test-merror:0.03950      

[6]	train-merror:0.06818	test-merror:0.08370                            

[7]	train-merror:0.06560	test-merror:0.08410                            

[8]	train-merror:0.06213	test-merror:0.07790                            

[9]	train-merror:0.07178	test-merror:0.08910                            

[10]	train-merror:0.07133	test-merror:0.08690                           

[11]	train-merror:0.07537	test-merror:0.09420                           

[12]	train-merror:0.07632	test-merror:0.09220                           

[13]	train-merror:0.08563	test-merror:0.10130                           

[14]	train-merror:0.09505	test-merror:0.10850                           

[15]	train-merror:0.09380	test-merror:0.11060                           

[16]	train-merror:0.08673	test-merror:0.10300                           

[17]	train-merror:0.08395	test-merror:0.10210                           

[18]	train-merror:0.08418	test-merror:0.10020                           

[19]	train-merror:0.08882	test-merror:

[10]	train-merror:0.79263	test-merror:0.79020                            

[11]	train-merror:0.79263	test-merror:0.79020                            

[12]	train-merror:0.79263	test-merror:0.79020                            

[13]	train-merror:0.79263	test-merror:0.79020                            

[14]	train-merror:0.79263	test-merror:0.79020                            

[15]	train-merror:0.79263	test-merror:0.79020                            

[16]	train-merror:0.79263	test-merror:0.79020                            

[17]	train-merror:0.79263	test-merror:0.79020                            

[18]	train-merror:0.79263	test-merror:0.79020                            

[19]	train-merror:0.79263	test-merror:0.79020                            

{'alpha': 3.8112493872306663, 'btype': 'Rn', 'colsample_bylevel': 0.581132407024442, 'colsample_bytree': 0.8281592125886286, 'eta': 0.24389242222880222, 'eval_metric': ('merror',), 'extra_dims': 10, 'gamma': 5.3683657447245876e-05, 'lambda': 0.008894

...

[0]	train-merror:0.04858	test-merror:0.07510                               

[1]	train-merror:0.01345	test-merror:0.05380                               

[2]	train-merror:0.00340	test-merror:0.04460                               

[3]	train-merror:0.00087	test-merror:0.03850                               

[4]	train-merror:0.00012	test-merror:0.03560                               

[5]	train-merror:0.00002	test-merror:0.03440                               

[6]	train-merror:0.00000	test-merror:0.03290                               

[7]	train-merror:0.00000	test-merror:0.03020                               

[8]	train-merror:0.00000	test-merror:0.02900                               

[9]	train-merror:0.00000	test-merror:0.02780                               

[10]	train-merror:0.00000	test-merror:0.02770                              

[11]	train-merror:0.00000	test-merror:0.02670                              

[12]	train-merror:0.00000	test-merror:0.02620                              


[3]	train-merror:0.17548	test-merror:0.20750                               

[4]	train-merror:0.16560	test-merror:0.20470                               

[5]	train-merror:0.17728	test-merror:0.20850                               

[6]	train-merror:0.18917	test-merror:0.22480                               

[7]	train-merror:0.21250	test-merror:0.23590                               

[8]	train-merror:0.24933	test-merror:0.28090                               

[9]	train-merror:0.25967	test-merror:0.28290                               

[10]	train-merror:0.27650	test-merror:0.29880                              

[11]	train-merror:0.28802	test-merror:0.31150                              

[12]	train-merror:0.32820	test-merror:0.34960                              

[13]	train-merror:0.30572	test-merror:0.31790                              

[14]	train-merror:0.33230	test-merror:0.34530                              

[15]	train-merror:0.31603	test-merror:0.32750                              


[6]	train-merror:0.01203	test-merror:0.05050                             

[7]	train-merror:0.01043	test-merror:0.04810                             

[8]	train-merror:0.00887	test-merror:0.04730                             

[9]	train-merror:0.00730	test-merror:0.04490                             

[10]	train-merror:0.00595	test-merror:0.04330                            

[11]	train-merror:0.00458	test-merror:0.04190                            

[12]	train-merror:0.00353	test-merror:0.04060                            

[13]	train-merror:0.00285	test-merror:0.03890                            

[14]	train-merror:0.00198	test-merror:0.03780                            

[15]	train-merror:0.00150	test-merror:0.03740                            

[16]	train-merror:0.00127	test-merror:0.03610                            

[17]	train-merror:0.00085	test-merror:0.03580                            

[18]	train-merror:0.00052	test-merror:0.03470                            

[19]	train-merror:0.00032

[11]	train-merror:0.00000	test-merror:0.02420                            

[12]	train-merror:0.00000	test-merror:0.02380                            

[13]	train-merror:0.00000	test-merror:0.02370                            

[14]	train-merror:0.00000	test-merror:0.02370                            

[15]	train-merror:0.00000	test-merror:0.02320                            

[16]	train-merror:0.00000	test-merror:0.02260                            

[17]	train-merror:0.00000	test-merror:0.02220                            

[18]	train-merror:0.00000	test-merror:0.02220                            

[19]	train-merror:0.00000	test-merror:0.02180                            

{'alpha': 0, 'btype': 'Rn', 'colsample_bylevel': 0.988925410524079, 'colsample_bytree': 0.8751327595513714, 'eta': 0.025752250741527413, 'eval_metric': ('merror',), 'extra_dims': 15, 'gamma': 0, 'lambda': 0.013513400718495553, 'max_depth': 8, 'min_child_weight': 0.00013832099125409472, 'num_class': 10, 'objective': 'multi:s

[15]	train-merror:0.09535	test-merror:0.09420                            

[16]	train-merror:0.09287	test-merror:0.09110                            

[17]	train-merror:0.09062	test-merror:0.08850                            

[18]	train-merror:0.08863	test-merror:0.08720                            

[19]	train-merror:0.08665	test-merror:0.08450                            

100%|██████████| 100/100 [7:05:38<00:00, 255.38s/trial, best loss: 0.0205]


In [4]:
print(best_val)
print(space_eval(spc, best))

0.0205
{'alpha': 0, 'btype': 'Rn', 'colsample_bylevel': 0.9691368851831579, 'colsample_bytree': 0.9513170996855516, 'eta': 0.12308954723938079, 'eval_metric': ('merror',), 'extra_dims': 15, 'gamma': 0, 'lambda': 0, 'max_depth': 10, 'min_child_weight': 0.0004484000405587484, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.6164874648683731}


In [13]:
from sklearn.neural_network import MLPClassifier

#mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4,
#                    solver='sgd', verbose=10, random_state=1,
#                    learning_rate_init=.1)
ms_params = {
    'hidden_layer_sizes' : [[50],
                            [100],
                            [50,50],
                            [100,100],
                            [200,200],
                            [200,200,200]
                           ],
    'max_iter':[400]
}
ms = GridSearchCV(MLPClassifier(), ms_params)
ms.fit(xtrain,y=ytrain)

NameError: name 'GridSearchCV' is not defined

In [ ]:
print(ms.cv_results_['params'][np.argmax(ms.cv_results_['mean_test_score'])])
max(ms.cv_results_['mean_test_score'])

In [ ]:
from sklearn.model_selection import GridSearchCV

gs_params = {
    'btype':['R','I','Rn','In'],
    'extra_dims':[0,64,128,256],
    'max_depth':[4],
    'eta':[0.1],
    'objective':['multi:softmax'],
    'subsample':[0.5],
    'colsample_bytree':[0.5],
    'n_estimators':[10],
    'min_child_weight':[1],
    'eval_metric':[['merror']],
    'num_class':[10]
}

gs = GridSearchCV(wxgbModel(), gs_params)
gs.fit(xtrain, y=ytrain)

In [ ]:
gs.cv_results_

In [ ]:
from wideboost.wrappers import wxgb

param = {'max_depth':2, 'objective':'multi:softmax','eval_metric':['merror'], 'num_class':10 }
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['eta'] = 0.001
param['btype'] = 'Rn'
param['extra_dims'] = 1024
param['lambda'] = 6
num_round = 200
#dtrain = xgb.DMatrix(xtrain,label=ytrain)
watchlist = [(dtrain,'train'),(dtest,'test')]
results = dict()
bst = wxgb.train(param, dtrain, num_round,watchlist,evals_result=results)



In [ ]:
plt.plot(results['train']['merror'])
plt.plot(results['test']['merror'])
plt.show()